In [2]:
# import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea
import requests
import json
from bs4 import BeautifulSoup as soup
import matplotlib.cm as cm
import matplotlib.colors as clr
%matplotlib inline
print('packages installed')

packages installed


In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: | 

In [ ]:
# retrieve neighbourhood data from Wikipedia and transform into readable dataframe

link = 'https://en.wikipedia.org/wiki/List_of_city-designated_neighbourhoods_in_Toronto'
ext = requests.get(link)
nh_data = soup(ext.content)
content = soup(str(nh_data))
nh_table = content.find('table',{'class':'wikitable sortable'})
str_table = str(nh_table.extract())

In [ ]:
# reading the table 

df = pd.read_html(str_table)[0]
df.head()

In [ ]:
list(df.columns)

In [ ]:
# drop the last 3 columns as they are not needed

df_drop = df.drop(['Neighbourhoods covered', 'Map', 'Unnamed: 5'], axis =1)

In [ ]:
df_drop

In [ ]:
df_drop.rename(columns={'City-designated neighbourhood' : 'Neighborhood', 'Former city/borough' : 'borough'}, inplace=True)

In [ ]:
df_drop.head()

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
df_data_1.drop(columns = ['Topic','Data Source','Category',], inplace=True)
df_data_1

In [ ]:
census_data = df_data_1.T
census_data

In [ ]:
new_header = census_data.iloc[3]
census_data = census_data[1:]
census_data.columns = new_header
census_data

In [ ]:
list(census_data.columns)

In [ ]:
census_data = census_data.iloc[4:]
census_data

In [ ]:
census_filtered = census_data.loc[:, ['Characteristic','Population, 2016','Working Age (25-54 years)','  After-tax income: Average amount ($)']]
census_filtered

In [ ]:
census_filtered.columns= ['Neighborhood', 'Population', 'Working_Age_Population','After_tax_income']
census_filtered

In [ ]:
#merge first two data sets of geo data and webscraped neighbourhoods

df_merge = pd.merge(df_drop, census_filtered, how = 'inner')
df_merge

In [ ]:
df_merge.dtypes

In [ ]:
df_merge['Population'] = df_merge['Population'].astype(str).astype(int)

In [ ]:
df_merge['Working_Age_Population'] = df_merge['Working_Age_Population'].astype(str).astype(int)

In [ ]:
df_merge['After_tax_income'] = df_merge['After_tax_income'].astype(str).astype(int)

In [ ]:
df_merge.dtypes

In [ ]:
pop_median = df_merge.Population.median()
pop_median

In [ ]:
Wap_median = df_merge.Working_Age_Population.median()
Wap_median

In [ ]:
Ati_median = df_merge.After_tax_income.median()
Ati_median

In [ ]:
df_merge['Pop_score'] = df_merge.Population/pop_median * 1
df_merge

In [ ]:
df_merge['Wap_score'] = df_merge.Population/Wap_median * 1.25
df_merge

In [ ]:
df_merge['Ati_score'] = df_merge.Population/Ati_median * 1.5
df_merge

In [ ]:
df_merge['Total_score'] = df_merge['Pop_score'] + df_merge['Wap_score'] + df_merge['Ati_score']
df_merge

In [ ]:
df_merge.sort_values(['Total_score'], inplace = True)
df_merge

In [ ]:
# retrieve neighbourhood data from Wikipedia and transform into readable dataframe

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
ext = requests.get(url)
nh_data = soup(ext.content)
content = soup(str(nh_data))
nh_table = content.find('table',{'class':'wikitable sortable'})
str_table = str(nh_table.extract())

In [ ]:
# reading the table 

df = pd.read_html(str_table)[0]
df.head()

In [ ]:
df_revised= df[df.Borough != 'Not assigned'].reset_index(drop=True)

In [ ]:
df_revised.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)

In [ ]:
#Clean the data out of extras by grouping

df_grouped = df_revised.groupby(['PostalCode','Borough'], as_index = False).agg(lambda x:','.join(x))
df_grouped.head()

In [ ]:
# get the latitudes and longitudes from dataframe given

geo_url = "https://cocl.us/Geospatial_data"

geo_df = pd.read_csv(geo_url)
geo_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
geo_df.head()

In [ ]:
df_code = pd.merge(df_grouped, geo_df, on='PostalCode')
df_code.head()
df_code.shape

In [ ]:
#yyz_core = ['West Toronto', 'East Toronto', 'Downtown Toronto', 'Central Toronto']
#df_yyz = df_code.loc[df_code['Borough'].isin(yyz_core)]
#df_yyz

In [ ]:
# Googled torontos coordinates as geo module does not work
yyz_lat = 43.6532
yyz_lng = -79.3832

In [ ]:
CLIENT_ID = 'EGX1TJDMIALSXJUGUIGAMIKR5BQGK1AZYBRGQLU2QSMVO32N' # your Foursquare ID
CLIENT_SECRET = 'FVRACNKHDALKLFXKXAIG5QLTAJYO3D4NO1W0ZDEXNL1SDI12' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df_yyz.dtypes

In [ ]:
df_code['Borough'] = df_code.Borough.astype(str)
df_code['Latitude'] = df_code.Latitude.astype(float)
df_code['Longitude'] = df_code.Longitude.astype(float)

In [ ]:
map_yyz = folium.Map(location=[yyz_lat, yyz_lng], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_code['Latitude'], df_code['Longitude'], df_code['PostalCode'], df_code['Borough'], df_code['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_yyz)
    
map_yyz

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(df_code['Latitude'], df_code['Longitude'], df_code['PostalCode'], df_code['Borough'], df_code['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
    

In [ ]:
v_df = pd.DataFrame(venues)
v_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(v_df.shape)
v_df.head()

In [ ]:
v_df.VenueCategory.describe

In [ ]:
v_df.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

In [ ]:
len(v_df['VenueCategory'].unique())

In [ ]:
# one hot encoding
yyz_central_onehot = pd.get_dummies(v_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
yyz_central_onehot['PostalCode'] = v_df['PostalCode'] 
yyz_central_onehot['Borough'] = v_df['Borough'] 
yyz_central_onehot['Neighborhoods'] = v_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(yyz_central_onehot.columns[-3:]) + list(yyz_central_onehot.columns[:-3])
yyz_central_onehot = yyz_central_onehot[fixed_columns]

print(yyz_central_onehot.shape)
yyz_central_onehot.head()

In [ ]:
yyz_central_freq = yyz_central_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(yyz_central_freq.shape)
yyz_central_freq.head()

In [ ]:
coffee = yyz_central_freq[["Neighborhoods","Coffee Shop"]]
coffee.head(10)
coffee.shape

In [ ]:
kclusters = 6

yyz_clustering = coffee.drop('Neighborhoods', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yyz_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
yyz_merged = coffee.copy()

# add clustering labels
yyz_merged["Cluster Labels"] = kmeans.labels_

In [ ]:
yyz_merged.head()

In [ ]:
yyz_merged = yyz_merged.rename(columns={'Neighborhoods':'Neighborhood'})
yyz_merged

In [ ]:
yyz_merged = yyz_merged.join(v_df.set_index("Neighborhood"), on="Neighborhood", how ='left')

print(yyz_merged.shape)
yyz_merged.head()

In [ ]:
yyz_merged.sort_values(["Cluster Labels"], inplace=True)
yyz_merged

In [ ]:
yyz_merged['VenueCategory'].value_counts()['Coffee Shop']

In [ ]:
# create map
yyz_clusters = folium.Map(location=[yyz_lat, yyz_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [clr.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(yyz_merged['BoroughLatitude'], yyz_merged['BoroughLongitude'], yyz_merged['Borough'], yyz_merged['PostalCode']
                                             ,yyz_merged['Neighborhood'], yyz_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(yyz_clusters)
       
yyz_clusters

In [ ]:
coffee["Cluster Labels"] = kmeans.labels_
coffee.head()

In [ ]:
objects = (1,2,3,4,5,6)
y_pos = np.arange(len(objects))
performance = coffee['Cluster Labels'].value_counts().to_frame().sort_index(ascending=True)
perf = performance['Cluster Labels'].tolist()
plt.bar(y_pos, perf, align='center', alpha=0.8, color=['red','blue','purple','silver','orange','green'])
plt.xticks(y_pos, objects)
plt.ylabel('No of Neighborhoods')
plt.xlabel('Cluster')
plt.title('How many Neighborhoods per Cluster')

plt.show()

In [ ]:
coffee['Cluster Labels'].value_counts()

In [ ]:
Neighborhoods = df_grouped[['Borough', 'Neighborhood']]

In [ ]:
Neighborhoods

In [ ]:
yyz_merged.dtypes

In [ ]:
zero_cluster = yyz_merged.loc[yyz_merged['Cluster Labels'] == 0]
df_cluster1 = pd.merge(Neighborhoods, zero_cluster, on='Neighborhood', how='left')
df_cluster1

In [ ]:
one_cluster =yyz_merged.loc[yyz_merged['Cluster Labels'] == 1]
df_cluster2 = pd.merge(Neighborhoods, one_cluster, on='Neighborhood')
df_cluster2

In [ ]:
two_cluster =yyz_merged.loc[yyz_merged['Cluster Labels'] == 2]
df_cluster3 = pd.merge(Neighborhoods, two_cluster, on='Neighborhood')
df_cluster3

In [ ]:
three_cluster =yyz_merged.loc[yyz_merged['Cluster Labels'] == 3]
df_cluster4 = pd.merge(Neighborhoods, three_cluster, on='Neighborhood')
df_cluster4

In [ ]:
four_cluster =yyz_merged.loc[yyz_merged['Cluster Labels'] == 4]
df_cluster5 = pd.merge(Neighborhoods, four_cluster, on='Neighborhood')
df_cluster5

In [ ]:
five_cluster =yyz_merged.loc[yyz_merged['Cluster Labels'] == 5]
df_cluster6 = pd.merge(Neighborhoods, five_cluster, on='Neighborhood')
df_cluster6

In [ ]:
clusters_mean = [df_cluster1['Coffee Shop'].mean(),df_cluster2['Coffee Shop'].mean(),df_cluster3['Coffee Shop'].mean(),
                 df_cluster4['Coffee Shop'].mean(),df_cluster5['Coffee Shop'].mean(),df_cluster6['Coffee Shop'].mean()]
objects = (1,2,3,4,5,6)
y_pos = np.arange(len(objects))
perf = clusters_mean
plt.bar(y_pos, perf, align='center', alpha=0.8, color=['red','blue','purple','silver','green','orange'])
plt.xticks(y_pos, objects)
plt.ylabel('Mean')
plt.xlabel('Cluster')
plt.title('Average number of Coffee Shops per Cluster')

plt.show()

In [ ]:
# one hot encoding
yyz_central_onehot_sbux = pd.get_dummies(v_df[['VenueName']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
yyz_central_onehot_sbux['PostalCode'] = v_df['PostalCode'] 
yyz_central_onehot_sbux['Borough'] = v_df['Borough'] 
yyz_central_onehot_sbux['Neighborhoods'] = v_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns_sbux = list(yyz_central_onehot_sbux.columns[-3:]) + list(yyz_central_onehot_sbux.columns[:-3])
yyz_central_onehot_sbux = yyz_central_onehot_sbux[fixed_columns_sbux]

print(yyz_central_onehot_sbux.shape)
yyz_central_onehot_sbux.head()

In [ ]:
yyz_central_freq_sbux = yyz_central_onehot_sbux.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(yyz_central_freq_sbux.shape)
yyz_central_freq_sbux.head()

In [ ]:
sbux = yyz_central_freq_sbux[["Neighborhoods","Starbucks"]]
sbux.head(10)
sbux.shape

In [ ]:
kclusters = 6

sbux_clustering = sbux.drop('Neighborhoods', 1)

kmeans_sbux = KMeans(n_clusters=kclusters, random_state=0).fit(sbux_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_sbux.labels_[0:10]

In [ ]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
sbux_merged = sbux.copy()

# add clustering labels
sbux_merged["Cluster Labels"] = kmeans_sbux.labels_

In [ ]:
sbux_merged = sbux_merged.rename(columns={'Neighborhoods':'Neighborhood'})
sbux_merged

In [ ]:
sbux_merged = sbux_merged.join(v_df.set_index("Neighborhood"), on="Neighborhood", how ='left')

print(sbux_merged.shape)
sbux_merged.head()

In [ ]:
sbux_merged.sort_values(["Cluster Labels"], inplace=True)
sbux_merged

In [ ]:
sbux_merged['VenueName'].value_counts()['Starbucks']

In [ ]:
sbux_merged

In [ ]:
sbux_merged['VenueName'] = sbux_merged['VenueName'].astype(str)
sbux_merged

In [ ]:
list(sbux_merged.VenueName)

In [ ]:
sbux_merged = sbux_merged.loc[sbux_merged['VenueName'].isin(['Starbucks'])]
sbux_merged

In [ ]:
# create map
sbux_clusters = folium.Map(location=[yyz_lat, yyz_lng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [clr.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(sbux_merged['BoroughLatitude'], sbux_merged['BoroughLongitude'], sbux_merged['Borough'], sbux_merged['PostalCode']
                                             ,sbux_merged['Neighborhood'], sbux_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(sbux_clusters)
       
sbux_clusters